## Setup and Import Libraries

In [24]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import time
import json

In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
client = OpenAI()

## Defining Custom Functions

In [4]:
def word_definition_quiz(word, definition_options):
    print("Welcome to the word definition quiz")
    print(f"What is the correct definition of this word: {word}")

    for num, option in enumerate(definition_options):
        print('\n')
        print(f'Definition #{num} is {option}')

    print('\n')

    num_choice = input("What is your choice? Return the #Num Option")
    return definition_options[int(num_choice)]

In [5]:
word = 'hello'
definition_options = ['greeting', 'farewell', 'sandwich']

In [6]:
word_definition_quiz(word=word, definition_options=definition_options)

Welcome to the word definition quiz
What is the correct definition of this word: hello


Definition #0 is greeting


Definition #1 is farewell


Definition #2 is sandwich




What is your choice? Return the #Num Option 0


'greeting'

## Convert Function to JSON Request

In [7]:
function_json = {
    "type": "function",
    "function": {
        "name": "word_definition_quiz",
        "parameters": {
            "type": "object",
            "properties": {
                "word": {
                    "type": "string",
                    "description": "A random word"
                },
                "definition_options": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "A Python list of strings of definitions, where only one is the correct definition for the word"
                }
            },
            "required": ["word", "definition_options"]
        }
    }
}

## Creating Asssitant

In [8]:
assistant_instructions = """ You help create a quiz for checking definitions of words, providing a word and then multiple
definition options, where only one option is correct
"""

assistant_name = "Word Definition Quiz Creator"

In [9]:
assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=assistant_instructions,
    model='gpt-3.5-turbo',
    tools=[function_json]
)

## Creating Thread and Message

In [10]:
thread = client.beta.threads.create()

In [11]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='Create a new quiz question word and definition list. Then let me know if the student answer received is correct',
)

## Creating Run

In [12]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        print(run.status)
        time.sleep(1)

    return run

In [13]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [14]:
wait_on_run(run=run, thread=thread)

in_progress
requires_action


Run(id='run_5dEa843BlgS7DhuWbkUA5BSv', assistant_id='asst_RUs2bZkxEZRSK8fSecduQaqV', cancelled_at=None, completed_at=None, created_at=1742338160, expires_at=1742338760, failed_at=None, incomplete_details=None, instructions=' You help create a quiz for checking definitions of words, providing a word and then multiple\ndefinition options, where only one option is correct\n', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_9Nfoh9alfmjUrwtmiG4syULn', function=Function(arguments='{"word":"meticulous","definition_options":["showing great attention to detail","lacking care or attention to detail","acting hastily without thought","showing preference for big picture over details"]}', name='word_definition_quiz'), type='function')]), type='submit_tool_outputs')

In [18]:
run = wait_on_run(run=run, thread=thread)

requires_action


In [19]:
run.status

'requires_action'

In [20]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]

In [22]:
name = tool_call.function.name
name

'word_definition_quiz'

In [23]:
arguments = tool_call.function.arguments
arguments

'{"word":"meticulous","definition_options":["showing great attention to detail","lacking care or attention to detail","acting hastily without thought","showing preference for big picture over details"]}'

In [25]:
arguments = json.loads(tool_call.function.arguments)
arguments

{'word': 'meticulous',
 'definition_options': ['showing great attention to detail',
  'lacking care or attention to detail',
  'acting hastily without thought',
  'showing preference for big picture over details']}

In [26]:
arguments['word']

'meticulous'

In [27]:
arguments['definition_options']

['showing great attention to detail',
 'lacking care or attention to detail',
 'acting hastily without thought',
 'showing preference for big picture over details']

In [29]:
response = word_definition_quiz(
    word=arguments['word'],
    definition_options=arguments['definition_options']
)

Welcome to the word definition quiz
What is the correct definition of this word: meticulous


Definition #0 is showing great attention to detail


Definition #1 is lacking care or attention to detail


Definition #2 is acting hastily without thought


Definition #3 is showing preference for big picture over details




What is your choice? Return the #Num Option 1


In [30]:
response

'lacking care or attention to detail'

In [31]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            'tool_call_id': tool_call.id,
            'output': json.dumps(response)
        }
    ]
)

In [32]:
run = wait_on_run(run, thread)

completed


In [34]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

The word is "meticulous" and here are the definition options:
1. Showing great attention to detail
2. Lacking care or attention to detail
3. Acting hastily without thought
4. Showing preference for big picture over details

The student answered with "Lacking care or attention to detail." 
This answer is incorrect. The correct definition for "meticulous" is "showing great attention to detail."


Create a new quiz question word and definition list. Then let me know if the student answer received is correct


